In [1]:
import requests
import pandas as pd
from pathlib import Path
import logging
import os
import sys
from scipy.stats import zscore
import dataframe_image as dfi

current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
from config.config import Config
from visualization.visualization import make_highlighted_table, team_styled_table

config = Config()

In [4]:
def overall_stats():
    """
    This function retrieves and merges playoff and regular season data.
    """
    # load data
    playoff_player_data = pd.read_parquet(f"../data/parquet/{config.playoff_player_data}")
    regular_player_data = pd.read_parquet(f"../data/parquet/{config.regular_player_data}")
    playoff_player_data = playoff_player_data.sort_values(by=['Dominance Quotient'], ascending=False).reset_index(drop=True)
    playoff_player_data.loc[9, "Player"] = ".Xero"
    

    # add playoff and regular prefixes
    playoff_player_data = playoff_player_data.add_prefix("playoff_")
    regular_player_data = regular_player_data.add_prefix("regular_")

    # merge data
    player_data = pd.merge(
        playoff_player_data,
        regular_player_data,
        left_on="playoff_Player",
        right_on="regular_Player",
        how="outer"
    )

    player_data["playoff_Dominance Quotient"] = player_data["playoff_Dominance Quotient"] * 0.6
    player_data["regular_Dominance Quotient"] = player_data["regular_Dominance Quotient"] * 0.4
    player_data["Total Dominance Quotient"] = player_data["playoff_Dominance Quotient"] + player_data["regular_Dominance Quotient"]

    final = player_data.sort_values(by=["Total Dominance Quotient"], ascending=False).reset_index(drop=True)
    final = final.drop(["regular_Player", "playoff_Dominance Quotient", "regular_Dominance Quotient"], axis=1)
    final = final.rename(columns={"playoff_Player": "Player"})
    final = final[["Player", "Total Dominance Quotient"] + [col for col in final.columns if col not in ["Player", "Total Dominance Quotient"]]] 
    final.index += 1

    final.to_parquet(f"../data/parquet/{config.overall_player_data}")

    return final

In [8]:
def team_styled_table(df: pd.DataFrame) -> pd.io.formats.style.Styler:
    # List of numerical columns to format
    numerical_columns = [
        'Total Dominance Quotient', 'playoff_Avg Score', 'playoff_Goals Per Game', 
        'playoff_Assists Per Game', 'playoff_Saves Per Game', 'playoff_Shots Per Game', 
        'playoff_Shooting %', 'playoff_Demos Inf. Per Game', 'playoff_Demos Taken Per Game', 
        'playoff_Big Boost Stolen', 'playoff_Small Boost Stolen', 'regular_Avg Score', 
        'regular_Goals Per Game', 'regular_Assists Per Game', 'regular_Saves Per Game', 
        'regular_Shots Per Game', 'regular_Shooting %', 'regular_Demos Inf. Per Game', 
        'regular_Demos Taken Per Game', 'regular_Big Boost Stolen', 'regular_Small Boost Stolen'
    ]
    
    # Dynamically filter numerical columns excluding 'Player'
    columns_to_format = [col for col in numerical_columns if col in df.columns and col != 'Player']
    
    # Apply formatting dynamically
    team_df = df.style.format({col: '{:.2f}' for col in columns_to_format}).set_table_styles([
        {'selector': 'thead th', 'props': 'color: #f8f8f2; background-color: #282a36;'},
        {'selector': 'tbody tr:nth-child(even) td, tbody tr:nth-child(even) th', 
            'props': 'background-color: #44475a; color: #f8f8f2;'},
        {'selector': 'tbody tr:nth-child(odd) td, tbody tr:nth-child(odd) th', 
            'props': 'background-color: #282a36; color: #f8f8f2;'},
        {'selector': 'td, th', 'props': 'border: none;'}
    ], overwrite=False)
    
    return team_df


In [9]:
df = overall_stats()
styled_table = team_styled_table(df)
styled_table

,Player,Total Dominance Quotient,playoff_Avg Score,playoff_Goals Per Game,playoff_Assists Per Game,playoff_Saves Per Game,playoff_Shots Per Game,playoff_Shooting %,playoff_Demos Inf. Per Game,playoff_Demos Taken Per Game,playoff_Big Boost Stolen,playoff_Small Boost Stolen,regular_Avg Score,regular_Goals Per Game,regular_Assists Per Game,regular_Saves Per Game,regular_Shots Per Game,regular_Shooting %,regular_Demos Inf. Per Game,regular_Demos Taken Per Game,regular_Big Boost Stolen,regular_Small Boost Stolen
1,jeridinho.,73.88,613.00,1.27,0.27,3.55,3.73,0.00,1.00,0.64,404.82,202.91,570.76,1.45,0.45,2.61,4.33,0.34,0.73,1.15,385.48,187.06
2,nyk,60.56,490.20,1.20,0.40,1.80,4.20,0.00,1.40,0.60,471.40,214.40,451.17,1.00,0.60,1.97,3.30,0.30,0.97,0.83,292.77,266.00
3,desi,60.36,481.75,0.75,0.58,1.92,4.17,0.00,1.83,0.75,299.08,318.83,561.00,1.35,0.35,2.39,3.55,0.38,1.29,0.84,319.42,220.58
4,Bax,60.20,485.56,0.56,1.11,2.44,3.44,0.00,1.22,1.11,248.11,244.78,443.32,0.84,0.84,1.94,3.45,0.24,0.90,1.00,345.90,233.10
5,Paperclip94,55.12,450.96,1.13,0.52,1.78,3.39,0.00,1.17,1.30,244.96,241.65,447.11,0.93,0.54,1.71,3.18,0.29,0.46,0.86,259.07,227.54
6,Duri,48.76,388.24,0.76,0.56,1.36,3.16,0.00,1.16,0.92,353.72,308.12,394.52,1.00,0.59,1.45,2.97,0.34,1.00,1.28,272.83,255.21
7,.Xero,45.12,361.08,0.68,0.68,1.32,2.84,0.00,1.48,1.16,424.20,247.16,343.38,1.03,0.55,0.83,2.93,0.35,1.07,1.07,269.93,247.72
8,buckystyle,44.88,401.80,0.60,0.50,2.00,3.50,0.00,1.50,1.00,427.60,236.20,353.67,0.96,0.26,1.19,3.30,0.29,1.11,1.33,402.81,249.11
9,Pullis.,44.20,427.20,0.40,0.80,2.40,2.40,0.00,0.80,2.20,289.40,251.80,332.57,0.75,0.57,1.50,2.07,0.36,0.61,1.14,210.39,208.93
10,Kyzeke,43.80,385.70,0.80,0.50,1.50,3.20,0.00,1.30,1.30,278.60,264.60,314.74,0.63,0.70,0.96,2.44,0.26,1.56,0.85,281.33,201.85
